In [491]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy import stats
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import torch
from torch import nn, optim
from classifier import nn_train, nn_predict, nn_accuracy
from torchvision import datasets, transforms, models
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and split the data

In [492]:
i = 1
x = np.load(f"numpy_data/A0{i}TX.npy")
x = x[:, :,1:]
x = np.swapaxes(x, 1, 2)
y = np.load(f"numpy_data/A0{i}TY.npy")
      
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)

print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)



final shapes are:  (230, 25, 313) (58, 25, 313) (230,) (58,)


In [493]:
train_coeff = featurize(X_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP() for _ in range(coeff_len)]
X_train = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x][:,:,:], y_train) for x  in range(coeff_len)),axis=-1)

test_coeff = featurize(X_test)
X_test = np.concatenate(tuple(csp[x].transform(test_coeff[x][:,:,:]) for x  in range(coeff_len)),axis=-1)

# X_train = torch.from_numpy(X_train)
# y_train = torch.from_numpy(y_train)

# X_test = torch.from_numpy(X_test)
# y_test = torch.from_numpy(y_test)
# print("final shapes are: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# model = nn.Sequential(nn.Linear(32, 64),
#                     nn.ReLU(),
#                     nn.Dropout(0.2),
#                     # nn.Linear(128, 64),
#                     # nn.ReLU(),
#                     # nn.Dropout(0.2),
#                     nn.Linear(64, 4),
#                     nn.LogSoftmax(dim=1))

# train_losses, validation_losses =\
#     nn_train(model, X_train, y_train, X_test, y_test, epochs=1000, lr=0.001, validate_every=100, debug=True)

# plt.plot(train_losses, label='Training loss')
# plt.plot(validation_losses, label='Validation loss')
# plt.legend(frameon=False)
# plt.show()


Computing rank from data with rank=None
    Using tolerance 58 (2.2e-16 eps * 25 dim * 1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 47 (2.2e-16 eps * 25 dim * 8.5e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 53 (2.2e-16 eps * 25 dim * 9.6e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 48 (2.2e-16 eps * 25 dim * 8.7e+15  max singular value)
    Estimated rank (ma

c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 25 (2.2e-16 eps * 25 dim * 4.6e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 23 (2.2e-16 eps * 25 dim * 4.2e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 24 (2.2e-16 eps * 25 dim * 4.3e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 25 (2.2e-16 eps * 25 dim * 4.6e+15  max singular value)
    Estimated rank (

In [494]:

# now we have CA6 + CD6 -> CD1 (6 matrices) * 7 features = 42 feature vector

scaler = StandardScaler().fit(X_train,y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

clf = pipeline.make_pipeline(svm.SVC())

# param_grid = [
#   {'C': [0.1, 0.5, 0.9, 1], 'kernel': ['linear']},
#   {'C': [0.1, 0.5, 0.9, 1], 'kernel': ['rbf']},
#  ]
# clf = GridSearchCV(svm.SVC(), param_grid)
# clf = KNeighborsClassifier(n_neighbors= 9)
# clf = QuadraticDiscriminantAnalysis()
clf.fit(X_train, y_train)
print(X_train.shape)


(230, 32)


In [495]:
print("Accuracy is ", sum(clf.predict(X_train) == y_train) / len(X_train))
print(len(X_train))

print("test Accuracy is ", sum(clf.predict(X_test) == y_test) / len(X_test))
print(len(X_test))


# c = pywt.WaveletPacket(x[0,0,:], 'db4', mode='symmetric', maxlevel=5)

# for node in c.get_level(5, 'natural'):
  # print(node.path)



Accuracy is  0.9695652173913043
230
test Accuracy is  0.603448275862069
58


In [496]:
# pywt.families()
# pywt.wavelist()
# x2 = np.load('numpy_data/A04TX.npy')
# x2 = x2[:, :,1:]
# x2 = np.swapaxes(x2, 1, 2)
# print(x2.shape, y.shape)
# y = np.load('numpy_data/A04TY.npy')

# test_coeff = featurize(x2)
# x2 = np.concatenate(tuple(csp[x].transform(test_coeff[x][:,:,:]) for x  in range(coeff_len)),axis=-1)

# print(clf.predict(x2))
# print("test Accuracy is ", sum(clf.predict(x2) == y) / len(x2))
# print(len(x2))



In [497]:
# print(y_train)